In [1]:
import torch
print(torch.__version__)                # 例: 2.1.0+cu118
print("CUDA available:", torch.cuda.is_available())
print("GPU count:", torch.cuda.device_count())
if torch.cuda.is_available():
    print("GPU name:", torch.cuda.get_device_name(0))


2.7.0+cu118
CUDA available: True
GPU count: 1
GPU name: NVIDIA GeForce RTX 4070 Laptop GPU


In [2]:
import os
import torch
import torch.nn.functional as F
from transformers import AutoImageProcessor, AutoModel
from time import time



# まずは画像読み込みとグローバル特徴量取得の関数を定義
def load_torch_image(fname, device=torch.device('cpu')):
    img = K.io.load_image(fname, K.io.ImageLoadType.RGB32, device=device)[None, ...]
    return img


# Must Use efficientnet global descriptor to get matching shortlists.
def get_global_desc(fnames, device = torch.device('cpu')):
    processor = AutoImageProcessor.from_pretrained('/kaggle/input/dinov2/pytorch/base/1')
    model = AutoModel.from_pretrained('/kaggle/input/dinov2/pytorch/base/1')
    model = model.eval()
    model = model.to(device)
    global_descs_dinov2 = []
    for i, img_fname_full in tqdm(enumerate(fnames),total= len(fnames)):
        key = os.path.splitext(os.path.basename(img_fname_full))[0]
        timg = load_torch_image(img_fname_full)
        with torch.inference_mode():
            inputs = processor(images=timg, return_tensors="pt", do_rescale=False).to(device)
            outputs = model(**inputs)
            dino_mac = F.normalize(outputs.last_hidden_state[:,1:].max(dim=1)[0], dim=1, p=2)
        global_descs_dinov2.append(dino_mac.detach().cpu())
    global_descs_dinov2 = torch.cat(global_descs_dinov2, dim=0)
    return global_descs_dinov2


def get_img_pairs_exhaustive(img_fnames):
    index_pairs = []
    for i in range(len(img_fnames)):
        for j in range(i+1, len(img_fnames)):
            index_pairs.append((i,j))
    return index_pairs


def get_image_pairs_shortlist(fnames,
                              sim_th = 0.6, # should be strict
                              min_pairs = 30,
                              exhaustive_if_less = 20,
                              device=torch.device('cpu')):
    num_imgs = len(fnames)
    if num_imgs <= exhaustive_if_less:
        return get_img_pairs_exhaustive(fnames)
    descs = get_global_desc(fnames, device=device)
    dm = torch.cdist(descs, descs, p=2).detach().cpu().numpy()
    # removing half
    mask = dm <= sim_th
    total = 0
    matching_list = []
    ar = np.arange(num_imgs)
    already_there_set = []
    for st_idx in range(num_imgs-1):
        mask_idx = mask[st_idx]
        to_match = ar[mask_idx]
        if len(to_match) < min_pairs:
            to_match = np.argsort(dm[st_idx])[:min_pairs]  
        for idx in to_match:
            if st_idx == idx:
                continue
            if dm[st_idx, idx] < 1000:
                matching_list.append(tuple(sorted((st_idx, idx.item()))))
                total+=1
    matching_list = sorted(list(set(matching_list)))
    return matching_list

def detect_aliked(img_fnames,
                  feature_dir = '.featureout',
                  num_features = 4096,
                  resize_to = 1024,
                  device=torch.device('cpu')):
    dtype = torch.float32 # ALIKED has issues with float16
    extractor = ALIKED(max_num_keypoints=num_features, detection_threshold=0.01, resize=resize_to).eval().to(device, dtype)
    if not os.path.isdir(feature_dir):
        os.makedirs(feature_dir)
    with h5py.File(f'{feature_dir}/keypoints.h5', mode='w') as f_kp, \
         h5py.File(f'{feature_dir}/descriptors.h5', mode='w') as f_desc:
        for img_path in tqdm(img_fnames):
            img_fname = img_path.split('/')[-1]
            key = img_fname
            with torch.inference_mode():
                image0 = load_torch_image(img_path, device=device).to(dtype)
                feats0 = extractor.extract(image0)  # auto-resize the image, disable with resize=None
                kpts = feats0['keypoints'].reshape(-1, 2).detach().cpu().numpy()
                descs = feats0['descriptors'].reshape(len(kpts), -1).detach().cpu().numpy()
                f_kp[key] = kpts
                f_desc[key] = descs
    return

def match_with_lightglue(img_fnames,
                   index_pairs,
                   feature_dir = '.featureout',
                   device=torch.device('cpu'),
                   min_matches=25,verbose=True):
    lg_matcher = KF.LightGlueMatcher("aliked", {"width_confidence": -1,
                                                "depth_confidence": -1,
                                                 "mp": True if 'cuda' in str(device) else False}).eval().to(device)
    with h5py.File(f'{feature_dir}/keypoints.h5', mode='r') as f_kp, \
        h5py.File(f'{feature_dir}/descriptors.h5', mode='r') as f_desc, \
        h5py.File(f'{feature_dir}/matches.h5', mode='w') as f_match:
        for pair_idx in tqdm(index_pairs):
            idx1, idx2 = pair_idx
            fname1, fname2 = img_fnames[idx1], img_fnames[idx2]
            key1, key2 = fname1.split('/')[-1], fname2.split('/')[-1]
            kp1 = torch.from_numpy(f_kp[key1][...]).to(device)
            kp2 = torch.from_numpy(f_kp[key2][...]).to(device)
            desc1 = torch.from_numpy(f_desc[key1][...]).to(device)
            desc2 = torch.from_numpy(f_desc[key2][...]).to(device)
            with torch.inference_mode():
                dists, idxs = lg_matcher(desc1,
                                         desc2,
                                         KF.laf_from_center_scale_ori(kp1[None]),
                                         KF.laf_from_center_scale_ori(kp2[None]))
            if len(idxs)  == 0:
                continue
            n_matches = len(idxs)
            if verbose:
                print (f'{key1}-{key2}: {n_matches} matches')
            group  = f_match.require_group(key1)
            if n_matches >= min_matches:
                 group.create_dataset(key2, data=idxs.detach().cpu().numpy().reshape(-1, 2))
    return

def import_into_colmap(img_dir, feature_dir ='.featureout', database_path = 'colmap.db'):
    db = COLMAPDatabase.connect(database_path)
    db.create_tables()
    single_camera = False
    fname_to_id = add_keypoints(db, feature_dir, img_dir, '', 'simple-pinhole', single_camera)
    add_matches(
        db,
        feature_dir,
        fname_to_id,
    )
    db.commit()
    return

c:\Users\337587\PlayGround\01_Personal\Kaggle\image-matching-challenge-2025\venv01\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## 本編

In [3]:
import os
import torch
import wandb
from lightglue import ALIKED, LightGlue
# 実験 ID
exp_num = "001"

# デバイス設定
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"running on {device}")

# W&B 初期化（ベースラインのハイパーパラメータを追加）
wandb.init(
    project="image-matching-challenge",
    name=f"exp_{exp_num}",
    config={
        "exp_num": exp_num,
        "device": str(device),
        # ショートリスト用
        "sim_th": 0.3,
        "min_pairs": 20,
        "exhaustive_if_less": 20,
        # 特徴抽出（ALIKED）用
        "num_features": 4096,
        "resize_to": 1024,
        "detection_threshold": 0.01,
        # LightGlueMatcher 用
        "width_confidence": -1,
        "depth_confidence": -1,
        "mp": True,  # CUDA 使用時に並列化
        # COLMAP マッピング設定
        "mapper_min_model_size": 3,
        "mapper_max_num_models": 25,
    }
)


c:\Users\337587\PlayGround\01_Personal\Kaggle\image-matching-challenge-2025\venv01\Lib\site-packages\lightglue\lightglue.py:24: FutureWarning: `torch.cuda.amp.custom_fwd(args...)` is deprecated. Please use `torch.amp.custom_fwd(args..., device_type='cuda')` instead.
  @torch.cuda.amp.custom_fwd(cast_inputs=torch.float32)


running on cuda


wandb: Currently logged in as: shaka066 (shaka066-personal) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


In [4]:
# 1) Kaggle API でデータをダウンロード＆展開
# DATA_DIR にダウンロード先を指定
DATA_DIR = r"C:\Users\337587\PlayGround\01_Personal\Kaggle\image-matching-challenge-2025\data"
os.makedirs(DATA_DIR, exist_ok=True)

# コマンドライン経由
#!kaggle competitions download -c image-matching-challenge-2025 -p {DATA_DIR}

# ZIP ファイルを展開
#import zipfile
#zip_path = os.path.join(DATA_DIR, "image-matching-challenge-2025.zip")
#with zipfile.ZipFile(zip_path, 'r') as z:
#    z.extractall(DATA_DIR)


# ダウンロード完了後、ディレクトリ構成を確認
#print("Downloaded files:")
#for root, dirs, files in os.walk(DATA_DIR):
#    level = root.replace(DATA_DIR, '').count(os.sep)
#    indent = ' ' * 2 * level
#    print(f"{indent}{os.path.basename(root)}/")
#    for f in files:
#        print(f"{indent}  - {f}")


In [5]:
# Kaggle → ローカル用に書き換え
DATA_ROOT =  r"C:\Users\337587\PlayGround\01_Personal\Kaggle\image-matching-challenge-2025\data"
TRAIN_DIR = os.path.join(DATA_ROOT, "train")
TEST_DIR  = os.path.join(DATA_ROOT, "test")

# サンプル提出ファイル
SAMPLE_SUB_CSV = os.path.join(DATA_ROOT, "sample_submission.csv")


In [10]:
# 結果＆モデル保存先
RESULTS_DIR = f"../../results/exp{exp_num}"
os.makedirs(RESULTS_DIR, exist_ok=True)


In [14]:
import os
import glob
import h5py
from time import time
from tqdm import tqdm

import wandb
import kornia.feature as KF

# ————————————————
# 定数・パス設定
# ————————————————
TRAIN_DIR   = "./data/image-matching-challenge-2025/train"
RESULTS_DIR = f"./results/exp{exp_num}"
# 出力フォルダが無ければ作成
os.makedirs(RESULTS_DIR, exist_ok=True)

# ————————————————
# 1) 画像パスの収集
# ————————————————
image_paths = sorted(glob.glob(os.path.join(TRAIN_DIR, "*", "*.*")))
wandb.log({"n_images": len(image_paths)})

# ————————————————
# 2) ショートリスト作成
# ————————————————
index_pairs = get_image_pairs_shortlist(
    image_paths,
    sim_th=wandb.config.sim_th,
    min_pairs=wandb.config.min_pairs,
    exhaustive_if_less=wandb.config.exhaustive_if_less,
    device=device
)
wandb.log({"n_pairs": len(index_pairs)})

# ————————————————
# 3) 特徴検出 (ALIKED)
# ————————————————
feature_dir = os.path.join(RESULTS_DIR, "featureout")
os.makedirs(feature_dir, exist_ok=True)

t0 = time()
detect_aliked(
    img_fnames= image_paths,
    feature_dir= feature_dir,
    num_features= wandb.config.num_features,
    resize_to=     wandb.config.resize_to,
    device=        device
)
wandb.log({"feature_detection_time": time() - t0})

# ————————————————
# 4) 特徴マッチング (LightGlue)
# ————————————————
t0 = time()
match_with_lightglue(
    img_fnames=   image_paths,
    index_pairs=  index_pairs,
    feature_dir=  feature_dir,
    device=       device,
    min_matches=  wandb.config.min_pairs,
    verbose=      False
)
wandb.log({"feature_matching_time": time() - t0})

# ————————————————
# 5) マッチ数を集計してログ
# ————————————————
matches_h5    = os.path.join(feature_dir, "matches.h5")
total_matches = 0
with h5py.File(matches_h5, "r") as f_match:
    for grp in f_match.values():
        for ds in grp.values():
            total_matches += ds.shape[0]
wandb.log({"n_total_matches": total_matches})
# ————————————————
# 6) 提出用 CSV の作成＆ W&B アーティファクト化
# ————————————————
import pandas as pd

# サンプル提出フォーマットを読み込む
# (上部で SAMPLE_SUB_CSV を定義済みのこと)
sub_df = pd.read_csv(SAMPLE_SUB_CSV)

# ── ここに、samples（もしくは predictions）から得られた
#     rotation_matrix, translation_vector を sub_df に書き込む処理を入れてください。
# 例:
# sub_df['rotation_matrix']    = my_rotations_list
# sub_df['translation_vector'] = my_translations_list

# ファイルとして保存
out_csv = os.path.join(RESULTS_DIR, "submission.csv")
sub_df.to_csv(out_csv, index=False)

# W&B にアーティファクトとしてアップロード
sub_art = wandb.Artifact(
    name=f"submission-exp{exp_num}",
    type="submission",
    description="Submission CSV for exp_" + exp_num
)
sub_art.add_file(out_csv)
wandb.log_artifact(sub_art)

print(f"Submission CSV saved to {out_csv}")


0it [00:00, ?it/s]


Loaded LightGlue model


0it [00:00, ?it/s]


Submission CSV saved to ./results/exp001\submission.csv
